# Defi-IA 2022 <a href="https://www.kaggle.com/c/defi-ia-2022/overview"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png" style="max-width: 70px; display: inline" alt="Python"/></a>
-------------------------------------------------------------

**Predict the accumulated daily rainfall on ground stations**  
*Quentin Douzery, Alexia Ghozland, Dario Moed*

## <font color = 'royalblue'>1 Imports</font>

In [1]:
##Packages
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn import linear_model

In [2]:
##Données
coords_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Other/stations_coordinates.csv' #Perso
#coords_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Other/stations_coordinates.csv' #INSA
#coords_fname  = '/kaggle/input/defi-ia-2022-test/Other/Other/stations_coordinates.csv'

xtrainS_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Train/X_station_train.csv' #Perso
#xtrainS_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/X_station_train.csv' #INSA
#fname = '/kaggle/input/defi-ia-2022-test/Train/Train/X_station_train.csv'

ytrain_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Train/Y_train.csv' #Perso
#ytrain_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/Y_train.csv' #INSA
#fname = '/kaggle/input/defi-ia-2022-test/Train/Train/Y_train.csv'

xtestS_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Test/X_station_test.csv' #Perso
#xtrainS_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/X_station_train.csv' #INSA

bl_obsT_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Test/Baselines/Baseline_observation_test.csv' #Perso

## <font color = 'royalblue'>2 Récupération des données</font>

In [3]:
##X_train
xtrainS = pd.read_csv(xtrainS_path,parse_dates=['date'],infer_datetime_format=True)
xtrainS['number_sta'] = xtrainS['number_sta'].astype('category')  
display(xtrainS)
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


Missing Data : 6899007 
 Shape is : (4409474, 9)


In [4]:
#Y_train
ytrain = pd.read_csv(ytrain_path, parse_dates=['date'], infer_datetime_format=True)
ytrain['number_sta'] = ytrain['number_sta'].astype('category')
display(ytrain)
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0
...,...,...,...,...
183742,2017-12-31,86137003,5.0,86137003_729
183743,2017-12-31,86165005,3.2,86165005_729
183744,2017-12-31,86272002,1.8,86272002_729
183745,2017-12-31,91200002,1.6,91200002_729


Missing Data : 21640 
 Shape is : (183747, 4)


In [5]:
##X_test
xtestS = pd.read_csv(xtestS_path,infer_datetime_format=True)
display(xtestS)
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22


Missing Data : 3914287 
 Shape is : (2304802, 8)


In [6]:
bl_obsT = pd.read_csv(bl_obsT_path,infer_datetime_format=True)
Id_pred = bl_obsT["Id"]

## <font color = 'royalblue'>3 Pré-processing</font>

In [7]:
def Get_StationDay (id):
    return id.split('_')[0] + "_" + id.split('_')[1]

### 3.1 Train data

**Remove NaNs**

In [8]:
xtrainS.dropna(inplace=True)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4


Missing Data : 0 
 Shape is : (2264105, 9)


In [9]:
ytrain.dropna(inplace=True)
display(ytrain.head())
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0


Missing Data : 0 
 Shape is : (162107, 4)


**Create StationDay column**

In [10]:
xtrainS = xtrainS.assign(StationDay=pd.Series(np.zeros(xtrainS.shape[0])).values)
xtrainS.loc[:,"StationDay"] = xtrainS.loc[:,"Id"].apply(Get_StationDay)

In [11]:
xtrainS = xtrainS.loc[xtrainS['StationDay'].isin(ytrain["Id"])]
xtrainS.reset_index(drop=True,inplace=True)
display(xtrainS.head())

,number_sta,date,ff,t,td,hu,dd,precip,Id,StationDay
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,14066001_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,14066001_0
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,14066001_0
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,14066001_0
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,14066001_0


**Drop useless variables**

In [12]:
xtrainS.drop(['number_sta', 'date', 'Id'], axis=1, inplace=True)

**Mean on 24 hours (except precip)**

In [13]:
xtrainS = xtrainS.groupby("StationDay", as_index=False).agg(pd.Series.sum, min_count = 24)

In [14]:
xtrainS.dropna(inplace=True)
xtrainS[['dd', 'hu', 'td', 't', 'ff']] = xtrainS[['dd', 'hu', 'td', 't', 'ff']].divide(24)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,StationDay,ff,t,td,hu,dd,precip
0,14066001_0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,14066001_1,8.041250,282.936667,279.997500,82.300000,205.625000,3.4
2,14066001_10,5.408750,279.557917,277.497917,86.750000,209.541667,6.0
3,14066001_100,4.296250,282.112917,277.944583,76.408333,134.958333,11.6
4,14066001_101,1.754583,282.805000,281.003333,88.745833,167.208333,5.6


Missing Data : 0 
 Shape is : (82120, 7)


**Clean Y_train**

In [15]:
ytrain = ytrain.loc[ytrain['Id'].isin(xtrainS['StationDay'])]

**Sort X_train**

In [16]:
xtrainS = xtrainS.set_index('StationDay')
xtrainS = xtrainS.reindex(index=ytrain['Id'])
xtrainS = xtrainS.reset_index()

**Drop useless variables**

In [17]:
ytrain.drop(['date', 'number_sta', 'Id'], axis=1, inplace=True)
xtrainS.drop(['Id'], axis=1, inplace=True)

In [18]:
xtrainS.reset_index(drop=True,inplace=True)
ytrain.reset_index(drop=True,inplace=True)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)
display(ytrain.head())
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,ff,t,td,hu,dd,precip
0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,4.759583,280.089583,278.888750,92.187500,158.666667,0.6
2,5.125833,280.076250,278.564167,90.308333,164.625000,0.2
3,4.868750,279.287500,278.418333,94.266667,167.625000,0.8
4,3.821250,281.128750,278.964583,86.545833,131.500000,0.0


Missing Data : 0 
 Shape is : (82120, 6)


,Ground_truth
0,3.4
1,4.0
2,10.0
3,3.6
4,10.3


Missing Data : 0 
 Shape is : (82120, 1)


### 3.2 Test data

**Create Stationday column**

In [20]:
xtestS = xtestS.assign(StationDay=pd.Series(np.zeros(xtestS.shape[0])).values)
xtestS.loc[:,"StationDay"] = xtestS.loc[:,"Id"].apply(Get_StationDay)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id,StationDay
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4,14047002_277
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5,14047002_277
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6,14047002_277
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7,14047002_277
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8,14047002_277


Missing Data : 3914287 
 Shape is : (2304802, 9)


**Drop useless variables**

In [21]:
xtestS.drop(['Id'], axis=1, inplace=True)

**Mean on 24 hours (except precipitations)**

In [22]:
xtestS = xtestS.groupby("StationDay", as_index=False).agg(pd.Series.sum, min_count = 24)
xtestS[['dd', 'hu', 'td', 't', 'ff']] = xtestS[['dd', 'hu', 'td', 't', 'ff']].divide(24)

**Clean X_test**

In [23]:
xtestS = xtestS.loc[xtestS['StationDay'].isin(bl_obsT["Id"])]
xtestS.reset_index(drop=True,inplace=True)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,StationDay,dd,hu,td,t,ff,precip,month
0,14047002_100,NaN,NaN,NaN,282.565000,NaN,9.9,288.0
1,14047002_223,NaN,NaN,NaN,285.415000,NaN,0.8,288.0
2,14047002_236,NaN,NaN,NaN,283.064167,NaN,7.0,288.0
3,14047002_256,NaN,NaN,NaN,283.611250,NaN,4.0,288.0
4,14047002_281,NaN,NaN,NaN,282.633750,NaN,0.0,288.0


Missing Data : 136607 
 Shape is : (85140, 8)


**Fill NaNs**

In [ ]:
#meandd=np.nanmean(Reg_predictions['dd'])
#meanhu=np.nanmean(Reg_predictions['hu'])
#meantd=np.nanmean(Reg_predictions['td'])
#meant=np.nanmean(Reg_predictions['t'])
#meanff=np.nanmean(Reg_predictions['ff'])

In [28]:
xtestS = xtestS.fillna(method="ffill")
xtestS = xtestS.fillna(method="backfill")
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,StationDay,dd,hu,td,t,ff,precip,month
0,14047002_100,206.75,89.125,277.68375,282.565000,3.37,9.9,288.0
1,14047002_223,206.75,89.125,277.68375,285.415000,3.37,0.8,288.0
2,14047002_236,206.75,89.125,277.68375,283.064167,3.37,7.0,288.0
3,14047002_256,206.75,89.125,277.68375,283.611250,3.37,4.0,288.0
4,14047002_281,206.75,89.125,277.68375,282.633750,3.37,0.0,288.0


Missing Data : 0 
 Shape is : (85140, 8)


**Sort X_test**

In [29]:
xtestS = xtestS.set_index('StationDay')
xtestS = xtestS.reindex(index=bl_obsT['Id'])
xtestS = xtestS.reset_index()

**Drop useless variables**

In [32]:
xtestS.drop(['Id', 'month'], axis=1, inplace=True)

KeyError: "['Id' 'month'] not found in axis"

**Rearrange columns**

In [33]:
xtestS = xtestS[['ff', 't', 'td', 'hu', 'dd', 'precip']]
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,ff,t,td,hu,dd,precip
0,5.237917,282.099583,279.107917,81.720833,224.625000,1.4
1,5.027500,280.895833,279.718750,92.291667,160.333333,1.0
2,5.547083,281.167917,278.469583,83.266667,218.458333,0.4
3,2.667083,280.804583,278.105417,83.345833,207.375000,0.8
4,4.724583,279.570417,271.835833,89.220833,163.583333,3.6


Missing Data : 0 
 Shape is : (85140, 6)


## <font color = 'royalblue'>4 Apprentissage</font>

In [34]:
method = linear_model.LinearRegression()
regLin = method.fit(xtrainS,ytrain)

## <font color = 'royalblue'>5 Prédictions</font>

In [35]:
predictions = regLin.predict(xtestS)

In [36]:
Reg_predictions = bl_obsT.copy()
Reg_predictions["Prediction"] = predictions
display(Reg_predictions.head())

,Id,Prediction
0,14066001_149,2.382006
1,14126001_149,2.167817
2,14137001_149,2.257604
3,14216001_149,1.721666
4,14296001_149,1.493103


## <font color = 'royalblue'>5 Post-processing</font>

In [37]:
Reg_predictions["Prediction"] = Reg_predictions["Prediction"] + 1
display(Reg_predictions.head())

,Id,Prediction
0,14066001_149,3.382006
1,14126001_149,3.167817
2,14137001_149,3.257604
3,14216001_149,2.721666
4,14296001_149,2.493103


## <font color = 'royalblue'>6 Exportation</font>



In [39]:
output_file = "Submissions_3_regression-lin-bis.csv"
Reg_predictions.to_csv('/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Submissions/' + output_file, index=False)